<a href="https://colab.research.google.com/github/Nubiancodingdelight/ACS-Project-Repository-/blob/main/Matrix_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#mounts google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!pip install pysam


In [29]:
# sets correct path
%cd drive/MyDrive/'ACS Research'/'Beginning Data'

[Errno 2] No such file or directory: 'drive/MyDrive/ACS Research/Beginning Data'
/content/drive/MyDrive/ACS Research/Beginning Data


In [30]:
#imports
import pysam
import re
import tensorflow as tf
import numpy as np
#from pyspark.sql import SparkSession

In [46]:
#parse each row
def matrix_row(CIG, SE, PO, INS):
  #intialize values here
  matrix_list= []
  label_list =[]
  bases = ['A','T','C','G']
  seq_list= list(SE)
  cigar_list = re.split(r'(\d+)', CIG)[1::]

  if len(cigar_list) ==2:
    #if it is a matched a sequence of 1's based on the base are added
    if cigar_list[1] == 'M':
      for item in seq_list:

        if int(PO) == int(INS):
            label_list.append([1,1,1,1])
        else:
            label_list.append([0,0,0,0])
        num_list = [0,0,0,0, PO]
        try:
            #if statement causes all values not in the base list to be ignored.
            num_list[bases.index(item)] = 1
            matrix_list.append(num_list)
        except ValueError:
            matrix_list.append(num_list)

            #skips rows with no match
      else:
        matrix_list.append([0,0,0,0, PO])
    else:
      for time in range(int(len(cigar_list)/2)):
        if cigar_list[(2*time)+1] == 'M':
          for item in seq_list:

            #generates label matrix
            if int(PO) == int(INS):
              label_list.append([1,1,1,1])
            else:
              label_list.append([0,0,0,0])

            num_list = [0,0,0,0, PO]

             #if statement causes all values not in the base list to be ignored.
            try:
              num_list[bases.index(item)] = 1
              matrix_list.append(num_list)
            except ValueError:
              matrix_list.append(num_list)

        #skips rows with no matches
        #add a statement to insert a blank row if there are no matches here
        else:
          matrix_list.append([0,0,0,0, PO])
  return matrix_list, label_list


In [47]:
#create matrix
samfile = pysam.AlignmentFile("bowtie2/SRR030257.bam", "rb")
final_matrix = []
label_matrix = []
for line in samfile:
  line_list = str(line).split("\t")
  #print(line_list)
  CIGAR = line_list[5]
  #print(CIGAR)
  SEQ = line_list[9]
  #print(SEQ)
  POS = line_list[3]
  INS = 950180
  #print(POS)
  #print(sum(matrix_row(CIGAR, SEQ, POS) , []))
  #final_matrix.append(matrix_row(CIGAR, SEQ, POS))
  final_matrix_temp, label_matrix_temp = matrix_row(CIGAR, SEQ, POS, INS)
  final_matrix += final_matrix_temp
  label_matrix += label_matrix_temp
  #final_matrix = sum(final_matrix, [])
  print(final_matrix)
  print(label_matrix)

  break

[[0, 1, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 1, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 0, 0, 1, '950180'], [0, 0, 0, 0, '950180']]
[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]

In [ ]:
def generate_matrix(filename, viral_pos):
  #create matrix
  samfile = pysam.AlignmentFile(filename, "rb")
  final_matrix = []
  label_matrix = []
  for line in samfile:
    line_list = str(line).split("\t")
    CIGAR = line_list[5]
    SEQ = line_list[9]
    POS = line_list[3]
    INS = viral_pos
    #calls function to generate a matrix and label for the corresponding row of sequence
    final_matrix_temp, label_matrix_temp = matrix_row(CIGAR, SEQ, POS, INS)
    final_matrix += final_matrix_temp
    label_matrix += label_matrix_temp
  return final_matrix, label_matrix


In [35]:
final_matrix

[[0, 1, 0, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [0, 0, 0, 1, '950180'],
 [0, 1, 0, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 0, 0, 1, '950180'],
 [0, 0, 1, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 0, 1, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 0, 0, 1, '950180'],
 [0, 1, 0, 0, '950180'],
 [1, 0, 0, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [0, 1, 0, 0, '950180'],
 [0, 0, 0, 1, '950180'],
 [0, 0, 0, 1, '950180'],
 [0, 0, 0, 0, '950180']]